In [1]:
import pandas as pd
import numpy as np

import helper
import project_helper
import project_tests

# Market Data

In [2]:
df = pd.read_csv('./data/eod-quotemedia.csv')

percent_top_dollar = 0.2
high_volume_symbols = project_helper.large_dollar_volume_stocks(df, 'adj_close', 'adj_volume', percent_top_dollar)
df = df[df['ticker'].isin(high_volume_symbols)]

close = df.reset_index().pivot(index='date', columns='ticker', values='adj_close')
volume = df.reset_index().pivot(index='date', columns='ticker', values='adj_volume')
dividends = df.reset_index().pivot(index='date', columns='ticker', values='dividends')

In [3]:
project_helper.print_dataframe(close)

# Part 1: Smart Beta Portfolio

## Index Weights

In [4]:
def generate_dollar_volume_weights(close, volume):
    """
    Generate dollar volume weights.

    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    volume : str
        Volume for each ticker and date

    Returns
    -------
    dollar_volume_weights : DataFrame
        The dollar volume weights for each ticker and date
    """
    assert close.index.equals(volume.index)
    assert close.columns.equals(volume.columns)
    
    product = close * volume
    
    result = product.copy()
    
    for index, row in result.iterrows():
        result.at[index] = row/np.sum(row)

    return result

In [5]:
project_tests.test_generate_dollar_volume_weights(generate_dollar_volume_weights)

Tests Passed


In [6]:
index_weights = generate_dollar_volume_weights(close, volume)
project_helper.plot_weights(index_weights, 'Index Weights')

FileNotFoundError: [Errno 2] No such file or directory: 'graphs/IndexWeights.html'